In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib


In [5]:
df = pd.read_csv(r"D:\phishing Detector\A_data\url_features_extracted1.csv")
#data = pd.read_csv(r"E:\ML project\Phishing-Detector\01-data\Phishing_Legitimate.csv")

In [4]:
print(data.head())
data.info()
data.describe()

                                               URL  url_length  \
0  https://keraekken-loagginnusa.godaddysites.com/          47   
1         https://metamsk01lgiix.godaddysites.com/          40   
2                          http://myglobaltech.in/          23   
3                   http://djtool-for-spotify.com/          30   
4  https://scearmcoommunnlty.com/invent/freind/get          47   

   has_ip_address  dot_count  https_flag  url_entropy  token_count  \
0               0          2           1     4.250669            6   
1               0          2           1     4.196439            6   
2               0          1           0     3.936180            5   
3               0          1           0     3.894740            5   
4               0          1           1     4.143127            7   

   subdomain_count  query_param_count  tld_length  path_length  \
0                1                  1           3            1   
1                1                  1           3 

,url_length,has_ip_address,dot_count,https_flag,url_entropy,token_count,subdomain_count,query_param_count,tld_length,path_length,has_hyphen_in_domain,number_of_digits,tld_popularity,suspicious_file_extension,domain_name_length,percentage_numeric_chars,ClassLabel
count,101219.000000,101219.000000,101219.000000,101219.000000,101219.000000,101219.000000,101219.000000,101219.000000,101219.000000,101219.000000,101219.000000,101219.000000,101219.000000,101219.000000,101219.000000,101219.000000,101218.000000
mean,35.060996,0.484879,3.007775,0.405645,3.976112,6.885733,1.511515,1.010591,5.003270,8.538249,0.042897,8.508669,0.328288,0.146613,6.292218,24.136487,0.370883
std,16.805678,0.499774,0.946612,0.491019,0.307068,1.670271,0.624755,0.202794,2.917429,14.701750,0.202626,8.540811,0.469593,0.353721,5.342714,22.490260,0.483043
min,7.000000,0.000000,0.000000,0.000000,2.521641,1.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,27.000000,0.000000,2.000000,0.000000,3.770942,5.000000,1.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
50%,31.000000,0.000000,3.000000,0.000000,3.937193,7.000000,2.000000,1.000000,3.000000,2.000000,0.000000,10.000000,0.000000,0.000000,3.000000,19.642857,0.000000
75%,35.000000,1.000000,4.000000,1.000000,4.100817,8.000000,2.000000,1.000000,8.000000,7.000000,0.000000,15.000000,1.000000,0.000000,10.000000,47.058824,1.000000
max,474.000000,1.000000,19.000000,1.000000,5.871503,48.000000,5.000000,14.000000,10.000000,317.000000,1.000000,164.000000,1.000000,1.000000,36.000000,65.957447,1.000000


In [6]:
df = df.dropna(subset=['ClassLabel'])
df['ClassLabel'] = df['ClassLabel'].astype(int)

# Shuffle dataset to avoid label ordering bias
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split features and labels
X = df.drop(['URL', 'ClassLabel'], axis=1)
y = df['ClassLabel']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Initialize Random Forest
model = RandomForestClassifier(
    n_estimators=150,          # number of trees
    max_depth=None,           # expand until all leaves are pure
    random_state=42,
    n_jobs=-1                 # use all CPU cores
)

# Train
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("✅ Accuracy:", round(accuracy_score(y_test, y_pred), 4))
print("\nClassification Report:\n", classification_report(y_test, y_pred))




✅ Accuracy: 0.9997

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     12736
           1       1.00      1.00      1.00      7508

    accuracy                           1.00     20244
   macro avg       1.00      1.00      1.00     20244
weighted avg       1.00      1.00      1.00     20244



In [8]:
sample_phish_1 = {
    'url_length': 45,
    'has_ip_address': 1,
    'dot_count': 3,
    'https_flag': 0,
    'url_entropy': 4.25,
    'token_count': 8,
    'subdomain_count': 1,
    'query_param_count': 1,
    'tld_length': 2,
    'path_length': 12,
    'has_hyphen_in_domain': 0,
    'number_of_digits': 6,
    'tld_popularity': 0,
    'suspicious_file_extension': 1,
    'domain_name_length': 9,
    'percentage_numeric_chars': 0.32
}

# ✅ Convert to DataFrame (with one row)
sample_df = pd.DataFrame([sample_phish_1])

# Predict
prediction = model.predict(sample_df)[0]

print("Prediction:", "Phishing" if prediction == 1 else "Legitimate")

Prediction: Legitimate


In [9]:
sample_phish_2 = {
    'url_length': 115,
    'has_ip_address': 0,
    'dot_count': 6,
    'https_flag': 1,
    'url_entropy': 5.8,
    'token_count': 18,
    'subdomain_count': 4,
    'query_param_count': 3,
    'tld_length': 4,
    'path_length': 45,
    'has_hyphen_in_domain': 1,
    'number_of_digits': 8,
    'tld_popularity': 0,
    'suspicious_file_extension': 1,
    'domain_name_length': 22,
    'percentage_numeric_chars': 0.25
}


sample_df = pd.DataFrame([sample_phish_2])

# Predict
prediction = model.predict(sample_df)[0]

print("Prediction:", "Phishing" if prediction == 1 else "Legitimate")

Prediction: Legitimate


In [8]:
legitimate_sample_1 = [[1, 25, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0]]
prediction_3 = model.predict(legitimate_sample_1)
print("Prediction for legitimate_sample_1:", prediction_3)

Prediction for legitimate_sample_1: [0]


c:\python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [9]:
legitimate_sample_2 = [[1, 30, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0]]

prediction_4 = model.predict(legitimate_sample_2)
print("Prediction for legitimate_sample_2:", prediction_4)

Prediction for legitimate_sample_2: [0]


c:\python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [11]:
accuracy = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test,y_pred)
cr = classification_report(y_test,y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", cr)

Accuracy: 0.982
Confusion Matrix:
 [[970  18]
 [ 18 994]]
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98       988
           1       0.98      0.98      0.98      1012

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000



In [11]:
import joblib

joblib.dump(model,filename=r"D:\phishing Detector\D_saved_models\rf_model_2.pkl")

['D:\\phishing Detector\\D_saved_models\\rf_model_2.pkl']